# BLEU

출처 : https://donghwa-kim.github.io/BLEU.html

- BLEU(Bilingual Evaluation Understudy) score란 성과지표로 데이터의 X가 순서정보를 가진 단어들(문장)로 이루어져 있고, y 또한 단어들의 시리즈(문장)로 이루어진 경우에 사용되며, 번역에 하는 모델에 주로 사용 된다

![calculation_bleu](img/bleu_1.png)

![description_bleu](img/bleu_2.png)

In [1]:
#n-gram 분석 https://blog.ilkyu.kr/entry/%EC%96%B8%EC%96%B4-%EB%AA%A8%EB%8D%B8%EB%A7%81-ngram

#sentence: 분석할 문장, num_gram: n-gram 단위
def word_ngram(sentence, num_gram):
    # in the case a file is given, remove escape characters
    sentence = sentence.replace('\n', ' ').replace('\r', ' ')
    text = tuple(sentence.split(' '))
    ngrams = [text[x:x+num_gram] for x in range(0, len(text)) if x+num_gram <= len(text)]
    return list(ngrams)

#n-gram 빈도 리스트 생성
def make_freqlist(ngrams):
    unique_ngrams = list(set(ngrams))
    freqlist = [0 for _ in range(len(unique_ngrams))]
    for ngram in ngrams:
        idx = unique_ngrams.index(ngram)
        freqlist[idx] +=1
    result = [unique_ngrams, freqlist]
    return result

 
# 두개 ngram 얼마나 겹치는지
def precision(output, target): # 
    result = 0
    output_len = 0
    for i in range(len(output[0])):
        if output[0][i] in target[0]:
            idx = target[0].index(output[0][i])
            result += min(output[1][i], target[1][idx])  # output frequency와 target frequency 중에서 min값 사용(Cliping)
    try:
        return result / sum(output[1])
    except ZeroDivisionError:
        return 0

In [4]:
sentence = '빛이 쐬는 노인은 완벽한 어두운곳에서 잠든 사람과 비교할 때 강박증이 심해질 기회가 훨씬 높았다'
a = word_ngram(sentence,1)
print(make_freqlist(a))

[[('때',), ('기회가',), ('훨씬',), ('노인은',), ('잠든',), ('완벽한',), ('높았다',), ('심해질',), ('비교할',), ('강박증이',), ('빛이',), ('사람과',), ('어두운곳에서',), ('쐬는',)], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [8]:
# predict = '빛이 쐬는 노인은 완벽한 어두운곳에서 잠든 사람과 비교할 때 강박증이 심해질 기회가 훨씬 높았다'
# true = '빛이 쐬는 노인은 완벽한 어둠에서 잠든 사람과 비교할 때 우울증이 심해질 가능성이 훨씬 높았다'
predict = 'The more decomposition the more flavor the food has'
true = 'The more the merrier I always say'

a = word_ngram(predict, 1)
a = make_freqlist(a)
b = word_ngram(true, 1)
b = make_freqlist(b)
precision(a,b)

0.3333333333333333

In [4]:
print(10/14)
print(5/13)
print(2/12)
print(1/11)

0.7142857142857143
0.38461538461538464
0.16666666666666666
0.09090909090909091


In [13]:
print(n_gram_precision(predict,true))
print(1*pow(10*5*2/14/13/12/11, 1/4))

0.7142857142857143
0.38461538461538464
0.16666666666666666
0.09090909090909091
0.25400289715190977
0.25400289715190977


![description2](img/bleu_3.png)

In [10]:
def n_gram_precision(sen_out, sen_tar):
    output = []
    target = []
    tar_freq = []
    sentence = sen_tar.replace('\n', ' ').replace('\r', ' ').split(' ')
    rouge_n = len(sentence) < 4
    if rouge_n:  # 문장 단어의 개수가 4개 미만일 때
        max_n = len(sentence) + 1
        rouge_list = []
    else:
        max_n = 5
    for i in range(1,max_n):
        n_gram = word_ngram(sen_out, i)
        out_tmp = make_freqlist(n_gram)
        output.append(out_tmp)
        n_gram2 = word_ngram(sen_tar, i)
        tar_tmp = make_freqlist(n_gram2)
        target.append(tar_tmp)
    result = 0
    n_pre = 0
    for i in range(len(output)):
        n_pre = precision(output[i], target[i])
        if rouge_n:
            print("ROUGE-" + str(i+1) + ": " + str(n_pre))
            rouge_list.append(n_pre)
        if i == 0:
            result = n_pre
        else:
            result *= n_pre
    if rouge_n:
        return rouge_list
    else:
        result = pow(result, 1/(max_n-1))
        # Brevity Penalty
        bp = min(1, sum(output[0][1])/sum(target[0][1]))
        return bp * result

- ROUGE-N: unigram, bigram, trigram 등 문장 간 중복되는 n-gram을 비교하는 지표입니다.

In [15]:
from torchtext.data.metrics import bleu_score
# candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
# references = 'It is a guide to action that ensures that the military will forever heed Party commands'
candidate = '나는 사람이다'
references = '나는 사람이다'
candidate_corpus = [candidate.split()]
references_corpus = [[references.split()]]

print(bleu_score(candidate_corpus, references_corpus))
print(n_gram_precision(candidate, references))

0.0
ROUGE-1: 1.0
ROUGE-2: 1.0
[1.0, 1.0]
